<a href="https://colab.research.google.com/github/whyumesh/-2022-23-Mini-Project-1A/blob/main/qwen2vl_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/huggingface/transformers accelerate flash_attn
!pip install qwen_vl_utils av

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-ifhh3rif
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-ifhh3rif
  Resolved https://github.com/huggingface/transformers to commit 238b13478df209ab534f2195a397dc64a3930883
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 80.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for transformers: filename=transformers-4.45.0.dev0-py3-none-any.whl size=9800484 sha256=7ef14d22a5a53769eb2646471ea6ef9ce66a9cec0ca33e80c822ffd92ab38c46
  Stored in directory: /tmp/pip-ephem-wheel-cache-ttcapk8t/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
  Created wheel for flash_attn: filename=flash_attn-2.6.3-cp310-cp310-linux_x86_64.whl size=187309225 sha256=237ef9c6157db394e1ddde

In [ ]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch

In [ ]:
model_name = "Qwen/Qwen2-VL-2B-Instruct"

In [ ]:
model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_name, torch_dtype=torch.bfloat16, attn_implementation="flash_attention_2", device_map="auto"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}


model.safetensors.index.json:   0%|          | 0.00/56.4k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/429M [00:00<?, ?B/s]

You are attempting to use Flash Attention 2.0 without specifying a torch dtype. This might lead to unexpected behaviour
`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/272 [00:00<?, ?B/s]

In [ ]:
processor = AutoProcessor.from_pretrained(model_name)

preprocessor_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "video",
                "video": "/content/code.mp4",
                "max_pixels": 360 * 420,
                "fps": 1.0,
            },
            {"type": "text", "text": "Explain the code highlighted with mouse cursor in blue color!"},
        ],
    }
]

In [ ]:
messages

[{'role': 'user',
  'content': [{'type': 'video',
    'video': '/content/code.mp4',
    'max_pixels': 151200,
    'fps': 1.0},
   {'type': 'text',
    'text': 'Explain the code highlighted with mouse cursor in blue color!'}]}]

In [ ]:
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

In [ ]:
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)

In [ ]:
inputs = inputs.to("cuda")

In [ ]:
generated_ids = model.generate(**inputs, max_new_tokens=512)

In [ ]:
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]

In [ ]:
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)

In [ ]:
print(output_text)

["The code you've highlighted is a Python script that appears to be related to image processing and possibly machine learning. Here's a breakdown of the code:\n\n1. **Importing Libraries**:\n   - `torch` and `torchvision` are used for working with PyTorch, a popular deep learning library.\n   - `torchvision.transforms` is used for image transformations.\n   - `torchvision.datasets` is used for loading and manipulating image datasets.\n   - `torchvision.models` is used for loading and manipulating models.\n   - `torchvision.utils` is used for utility functions related to image processing.\n\n2. **Helper Functions**:\n   - There are several helper functions that are not fully visible in the screenshot. These functions are likely used for specific tasks such as image processing, model loading, or data manipulation.\n\n3. **Main Code**:\n   - The main code block is written in Python and uses the `torch` library to perform image processing tasks.\n   - The code starts by importing necessary